In [1]:
import hr_estimate
from load_data import *
from hr_estimate import *
from load_data import *
from collections import Counter
import importlib
importlib.reload(hr_estimate)

<module 'hr_estimate' from 'D:\\Rice_25spring\\594\\Jiayi_Gao\\noise_detection\\hr_estimate.py'>

In [2]:
import os
import csv

# Set your base directory (the one containing the subdirectories)
base_dir = "./data/physionet_data/test_data"

data_num=0
hr_est_err=0
# Iterate through subdirectories
for subdir in os.listdir(base_dir):
    subdir_path = os.path.join(base_dir, subdir)
    if os.path.isdir(subdir_path):
        # Find the first .tsv file in the subdirectory
        for file in os.listdir(subdir_path):
            if file.endswith(".tsv"):
                tsv_path = os.path.join(subdir_path, file)
                try:
                    with open(tsv_path, 'r', newline='') as tsvfile:
                        reader = list(csv.reader(tsvfile, delimiter='\t'))
                        if reader:
                            first_line = reader[0]
                            last_line = reader[-1]
                            seg_start = first_line[1] if len(first_line) > 1 else "N/A"
                            seg_end = last_line[0] if len(last_line) > 0 else "N/A"
                            seg_dur=float(seg_end)-float(seg_start)
                            count=Counter()
                            for line in reader:
                                if line:
                                    seg_num=line[-1]
                                    if int(seg_num) != 0:
                                        count[int(seg_num)]+=1
                            for i in range(1, 5):
                                count.setdefault(i, 0)

                            num_cycle = min(count.values())
                            hr_cycle_low=seg_dur/num_cycle-0.1
                            hr_cycle_high=seg_dur/num_cycle+0.1
                            print(f"approximate heart cycle duration {seg_dur/num_cycle} seconds")
                            fs,sig=load_wav(f"{subdir_path}/{subdir}_PV.wav")
                            data_num=data_num+1
                            #fs,sig=down_sampling(sig,fs,target_fs)
                            sig=pre_process(sig)
                            est_hr=pre_find_hr(sig,fs)
                            if est_hr > hr_cycle_high or est_hr < hr_cycle_low:
                                hr_est_err+=1
                                print(f"est hr cycle: {est_hr:.2f} seconds")
                        else:
                            print(f"{subdir}: File is empty")

                except Exception as e:
                    print(f"Error reading {tsv_path}: {e}")

                break  # only the first .tsv file
print(data_num,hr_est_err)

approximate heart cycle duration 0.5696296296296297 seconds
est hr cycle: 0.96 seconds
approximate heart cycle duration 0.5963118333333333 seconds
est hr cycle: 1.05 seconds
approximate heart cycle duration 0.8084210526315789 seconds
est hr cycle: 1.09 seconds
approximate heart cycle duration 0.6293904285714286 seconds
est hr cycle: 0.90 seconds
approximate heart cycle duration 0.5357474285714285 seconds
est hr cycle: 0.85 seconds
approximate heart cycle duration 0.7640869565217392 seconds
approximate heart cycle duration 0.4730054999999999 seconds
est hr cycle: 0.93 seconds
approximate heart cycle duration 0.6066666666666667 seconds
est hr cycle: 1.06 seconds
approximate heart cycle duration 0.5016785714285714 seconds
est hr cycle: 0.80 seconds
approximate heart cycle duration 0.5693127499999999 seconds
est hr cycle: 0.89 seconds
approximate heart cycle duration 0.6630776666666667 seconds
est hr cycle: 0.92 seconds
approximate heart cycle duration 0.6164345555555555 seconds
Error read